This notebook checks what MCGs are available for the following districts to determine level of suppression:

EH12, L4, CF10, TW2, HA9

In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
sys.path.append(f'{project_path}/ft_events/src/utils')

from google.cloud import bigquery
from google.cloud import storage

import importlib

import numpy as np
import pandas as pd
from plotly import graph_objs as go
import seaborn as sns
import geopandas as gpd

import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px
import ipywidgets as widgets

from fintrans_toolbox.src import table_utils as t
from fintrans_toolbox.src import bq_utils as bq



client = bigquery.Client()

In [ ]:
postal_districts = ('EH12', 'CF10', 'HA9', 'L4', 'TW2')

-----------------------

-----------------------

-----------------------

## Spend merchant location

In [ ]:
sql_sml_spend = f"""SELECT *
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
mcc = 'All' AND
merchant_location_level = 'POSTAL_DISTRICT' AND
merchant_location IN {postal_districts}
ORDER BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders"""

sml_df = client.query(sql_sml_spend).to_dataframe()
sml_df = t.create_date_time(sml_df)


In [ ]:
for i in postal_districts:
    
    fig = px.line(
    sml_df[(sml_df['merchant_location'] == i) & (sml_df['cardholder_issuing_level'] == 'All')],
    x="date_time",
    y="spend",
    color = 'mcg',
    template='simple_white',
    height = 500,
    width = 800,
    title = f'Raw spend at {i}')

    fig.show()

In [ ]:
for i in postal_districts:
    
    fig = px.line(
    sml_df[(sml_df['merchant_location'] == i) & (sml_df['cardholder_issuing_level'] == 'All')],
    x="date_time",
    y="spend",
    color = 'mcg',
    template='simple_white',
    height = 500,
    width = 800,
    title = f'Raw spend at {i}')

    fig.show()

In [ ]:

for i in postal_districts:
    fig = px.bar(
        sml_df[(sml_df['merchant_location'] == i) & (sml_df['cardholder_issuing_level'] == 'All')],
        x="date_time",
        y="spend",
        color='mcg',
        template='simple_white',
        height=500,
        width=800,
        title=f'Raw spend at {i}'
    )
    fig.show()


### EH12: 

#### Vehicle rental contribution

In [ ]:
sml_df
# 'all', 'sum of all mcgs inc vechicle rental', 'sum all mcgs exc vech rental')

eh12_all = sml_df[(sml_df['merchant_location'] == 'EH12') & (sml_df['mcg'] == 'All') & (sml_df['cardholder_issuing_level'] == 'All')].copy()
eh12_all_summed = sml_df[(sml_df['merchant_location'] == 'EH12') & (sml_df['mcg'] != 'All') & (sml_df['cardholder_issuing_level'] == 'All')].groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()
eh12_all_summed_veh_rental_excluded = sml_df[(sml_df['merchant_location'] == 'EH12') & (sml_df['mcg'] != 'All') & (sml_df['mcg'] != 'VEHICLE RENTAL') & (sml_df['cardholder_issuing_level'] == 'All')].groupby('date_time').agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()

eh12_all['filter'] = 'All'
eh12_all_summed['filter'] = 'MCGs summed'
eh12_all_summed_veh_rental_excluded['filter'] = 'MCGs summed exc vehicle rental'

final_eh12 = pd.concat([eh12_all, eh12_all_summed, eh12_all_summed_veh_rental_excluded]).copy()


In [ ]:
fig = px.line(
    final_eh12,
    x="date_time",
    y="spend",
    color='filter',
    template='simple_white',
    height=500,
    width=1000,
    title=f'Raw spend at EH12'
)
fig.show()

#### Number of mcgs

In [ ]:
eh12_all_mcgs = sml_df[(sml_df['merchant_location'] == 'EH12')  & (sml_df['cardholder_issuing_level'] == 'All')].copy()

In [ ]:
num_mcgs = eh12_all_mcgs.groupby('date_time')['mcg'].nunique().reset_index()
num_mcgs.rename(columns={'mcg': 'num_mcgs'}, inplace=True)
fig = px.line(
    num_mcgs,
    x="date_time",
    y="num_mcgs",
    template='simple_white',
    height=500,
    width=700,
    title=f'Number of unique mcgs at EH12'
)
fig.show()

#### June 2025 mcgs

In [ ]:
eh12_all_mcgs[eh12_all_mcgs['time_period_value'] == '202506']

#### Country spend at EH12

In [ ]:
eh12_international = sml_df[(sml_df['cardholder_issuing_country'] != 'All') & (sml_df['mcg'] == 'All') & (sml_df['merchant_location'] == 'EH12')].copy()

fig = px.line(
    eh12_international,
    x="date_time",
    y="spend",
    color = 'cardholder_issuing_country',
    template='simple_white',
    height=500,
    width=700,
    title='Country spend at EH12'
)
fig.show()

In [ ]:
eh12_international[eh12_international['time_period_value'] == '202506']

In [ ]:
num_countries = eh12_international.groupby('date_time')['cardholder_issuing_country'].nunique().reset_index()
fig = px.line(
    num_countries,
    x="date_time",
    y="cardholder_issuing_country",
    template='simple_white',
    height=500,
    width=700,
    title=f'Number of unique countries spending at EH12'
)
fig.show()

--------------------

--------------------

## Retail table

------------

In [ ]:
sql_ret_district = f"""SELECT time_period_value,
merchant_location, cardholder_location,  mcg, spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns
WHERE time_period = 'Month' AND
mcg = 'All' AND
merchant_location IN {postal_districts} AND
cardholder_location_level = 'POSTAL_DISTRICT'
ORDER BY time_period_value,  merchant_location, cardholder_location, mcg, spend, transactions, cardholders"""
district_ret = client.query(sql_ret_district).to_dataframe()
district_ret = t.create_date_time(district_ret)

In [ ]:
district_ret = district_ret.groupby(['date_time', 'merchant_location']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()

In [ ]:
fig = px.line(
district_ret,
x="date_time",
y="spend",
color = 'merchant_location',
template='simple_white',
height = 500,
width = 800,
title = 'Country spend at postal districts')
fig.update_layout(title_subtitle_text=f'mcg = "All"')

fig.show()

---------------------------

---------------------------

## SPOC table

-----------------

In [ ]:
sql_spoc_district = f"""SELECT *
FROM ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel
WHERE time_period = 'Month' AND
mcg = 'All' AND
cardholder_origin_country = 'POSTAL_DISTRICT' AND
destination_country = 'UNITED KINGDOM'
ORDER BY time_period_value,  cardholder_location, mcg, spend, transactions, cardholders"""
district_spoc = client.query(sql_spoc_district).to_dataframe()
district_spoc = t.create_date_time(district_spoc)

In [ ]:
district_spoc = district_spoc.groupby(['date_time', 'merchant_channel']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()

In [ ]:
fig = px.line(
district_spoc,
x="date_time",
y="spend",
color = 'merchant_channel',
template='simple_white',
height = 500,
width = 800,
title = 'United Kingdom spend by channel')
fig.update_layout(title_subtitle_text=f'mcg = "All"')

fig.show()